**Troubleshooting Scripts - Plan Cache-Based Execution Statistics**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Query Execution Statistics (sys.dm\_exec\_query\_stats)**

Simple version without aggregation by similar queries. May return more than one row per statement Sort results based on your optimization targets (IO, CPU, etc).

Pay attention to time when plan was cached (Cached Time, Last Exec Time) to estimate the impact. It also may make sense to sort by _Total_ and _AVG_ metrics and cross-check the results in your analysis.

Some columns in the output may not work in older SQL Server versions.

In [ ]:
;WITH Queries
AS
(
	SELECT TOP 50
		qs.creation_time AS [Cached Time]
		,qs.last_execution_time AS [Last Exec Time]
		,qs.execution_count AS [Exec Cnt]
		,CONVERT(DECIMAL(10,5),
			IIF(datediff(second,qs.creation_time, qs.last_execution_time) = 0,
				NULL,
				1.0 * qs.execution_count / 
					datediff(second,qs.creation_time, qs.last_execution_time)
			)
		) AS [Exec Per Second]
		,(qs.total_logical_reads + qs.total_logical_writes) / 
			qs.execution_count AS [Avg IO]
		,(qs.total_worker_time / qs.execution_count / 1000) 
			AS [Avg CPU(ms)]
		,qs.total_logical_reads AS [Total Reads]
		,qs.last_logical_reads AS [Last Reads]
		,qs.total_logical_writes AS [Total Writes]
		,qs.last_logical_writes AS [Last Writes]
		,qs.total_worker_time / 1000 AS [Total Worker Time]
		,qs.last_worker_time / 1000 AS [Last Worker Time]
		,qs.total_elapsed_time / 1000 AS [Total Elapsed Time]
		,qs.last_elapsed_time / 1000 AS [Last Elapsed Time]
		,qs.total_rows AS [Total Rows] 
		,qs.last_rows AS [Last Rows] 
		,qs.total_rows / qs.execution_count AS [Avg Rows]
		,qs.total_physical_reads AS [Total Physical Reads]
		,qs.last_physical_reads AS [Last Physical Reads]
		,qs.total_physical_reads / qs.execution_count 
			AS [Avg Physical Reads]
		,qs.total_grant_kb AS [Total Grant KB]
		,qs.last_grant_kb AS [Last Grant KB]
		,(qs.total_grant_kb / qs.execution_count) 
			AS [Avg Grant KB] 
		,qs.total_used_grant_kb AS [Total Used Grant KB]
		,qs.last_used_grant_kb AS [Last Used Grant KB]
		,(qs.total_used_grant_kb / qs.execution_count) 
			AS [Avg Used Grant KB] 
		,qs.total_ideal_grant_kb AS [Total Ideal Grant KB]
		,qs.last_ideal_grant_kb AS [Last Ideal Grant KB]
		,(qs.total_ideal_grant_kb / qs.execution_count) 
			AS [Avg Ideal Grant KB] 
		,qs.total_columnstore_segment_reads AS [Total CSI Segments Read] -- SQL Server 2016 SP2+
		,qs.last_columnstore_segment_reads 	AS [Last CSI Segments Read] -- SQL Server 2016 SP2+
		,(qs.total_columnstore_segment_reads / qs.execution_count) AS [AVG CSI Segments Read] -- SQL Server 2016 SP2+
		,qs.max_dop AS [Max DOP]
		,qs.total_spills AS [Total Spills] -- SQL Server 2016 SP2+
		,qs.last_spills AS [Last Spills] -- SQL Server 2016 SP2+
		,(qs.total_spills / qs.execution_count) AS [Avg Spills] -- SQL Server 2016 SP2+
		,qs.statement_start_offset
		,qs.statement_end_offset
		,qs.plan_handle
		,qs.sql_handle
	FROM 
		sys.dm_exec_query_stats qs WITH (NOLOCK)
	ORDER BY
		[Avg IO] DESC
)
SELECT
	SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
	((
		CASE qs.statement_end_offset
			WHEN -1 THEN DATALENGTH(qt.text)
			ELSE qs.statement_end_offset
		END - qs.statement_start_offset)/2)+1) AS SQL
	,TRY_CONVERT(xml,qp.query_plan) AS [Query Plan]
	,qs.*
FROM 
	Queries qs
		OUTER APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
		OUTER APPLY 
				sys.dm_exec_text_query_plan
				(
					qs.plan_handle
					,qs.statement_start_offset
					,qs.statement_end_offset
				) qp
OPTION (RECOMPILE, MAXDOP 1);

**Query Execution Statistics (sys.dm\_exec\_query\_stats) with Query Cost**

Same query as above with query cost. Use with care as the parsing of plan XML is expensive and time consuming.

Some columns in the output may not work in older SQL Server versions.

In [ ]:
;WITH XMLNAMESPACES(DEFAULT 'http://schemas.microsoft.com/sqlserver/2004/07/showplan')
,Queries
AS
(
	SELECT TOP 50
		qs.creation_time AS [Cached Time]
		,qs.last_execution_time AS [Last Exec Time]
		,qs.execution_count AS [Exec Cnt]
		,CONVERT(DECIMAL(10,5),
			IIF(datediff(second,qs.creation_time, qs.last_execution_time) = 0,
				NULL,
				1.0 * qs.execution_count / 
					datediff(second,qs.creation_time, qs.last_execution_time)
			)
		) AS [Exec Per Second]
		,(qs.total_logical_reads + qs.total_logical_writes) / 
			qs.execution_count AS [Avg IO]
		,(qs.total_worker_time / qs.execution_count / 1000) 
			AS [Avg CPU(ms)]
		,qs.total_logical_reads AS [Total Reads]
		,qs.last_logical_reads AS [Last Reads]
		,qs.total_logical_writes AS [Total Writes]
		,qs.last_logical_writes AS [Last Writes]
		,qs.total_worker_time / 1000 AS [Total Worker Time]
		,qs.last_worker_time / 1000 AS [Last Worker Time]
		,qs.total_elapsed_time / 1000 AS [Total Elapsed Time]
		,qs.last_elapsed_time / 1000 AS [Last Elapsed Time]
		,qs.total_rows AS [Total Rows] 
		,qs.last_rows AS [Last Rows] 
		,qs.total_rows / qs.execution_count AS [Avg Rows]
		,qs.total_physical_reads AS [Total Physical Reads]
		,qs.last_physical_reads AS [Last Physical Reads]
		,qs.total_physical_reads / qs.execution_count 
			AS [Avg Physical Reads]
		,qs.total_grant_kb AS [Total Grant KB]
		,qs.last_grant_kb AS [Last Grant KB]
		,(qs.total_grant_kb / qs.execution_count) 
			AS [Avg Grant KB] 
		,qs.total_used_grant_kb AS [Total Used Grant KB]
		,qs.last_used_grant_kb AS [Last Used Grant KB]
		,(qs.total_used_grant_kb / qs.execution_count) 
			AS [Avg Used Grant KB] 
		,qs.total_ideal_grant_kb AS [Total Ideal Grant KB]
		,qs.last_ideal_grant_kb AS [Last Ideal Grant KB]
		,(qs.total_ideal_grant_kb / qs.execution_count) AS [Avg Ideal Grant KB] 
		,qs.total_columnstore_segment_reads	AS [Total CSI Segments Read] -- SQL Server 2016 SP2+
		,qs.last_columnstore_segment_reads 	AS [Last CSI Segments Read] -- SQL Server 2016 SP2+
		,(qs.total_columnstore_segment_reads / qs.execution_count) AS [AVG CSI Segments Read] -- SQL Server 2016 SP2+
		,qs.max_dop AS [Max DOP]
		,qs.total_spills AS [Total Spills] -- SQL Server 2016 SP2+
		,qs.last_spills AS [Last Spills] -- SQL Server 2016 SP2+
		,(qs.total_spills / qs.execution_count) AS [Avg Spills] -- SQL Server 2016 SP2+
		,qs.statement_start_offset
		,qs.statement_end_offset
		,qs.plan_handle
		,qs.sql_handle
	FROM 
		sys.dm_exec_query_stats qs WITH (NOLOCK)
	ORDER BY
		[Avg IO] DESC
)
SELECT
	SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
	((
		CASE qs.statement_end_offset
			WHEN -1 THEN DATALENGTH(qt.text)
			ELSE qs.statement_end_offset
		END - qs.statement_start_offset)/2)+1) AS SQL
	,qp_xml.[Query Plan]
	,p.[Query Cost]
	,qs.*
FROM 
	Queries qs
		OUTER APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
		OUTER APPLY 
				sys.dm_exec_text_query_plan
				(
					qs.plan_handle
					,qs.statement_start_offset
					,qs.statement_end_offset
				) qp
		OUTER APPLY
		(
			SELECT TRY_CONVERT(xml,qp.query_plan) AS [Query Plan]
		) qp_xml
		OUTER APPLY 
		(
			SELECT
				p.n.value(N'@EstimatedTotalSubtreeCost', N'FLOAT') 
					AS [Query Cost]
			FROM
				qp_xml.[Query Plan].nodes(N'//RelOp') p(n)
			WHERE
				p.n.value(N'@NodeId', N'INT') = 0
		) p 
OPTION (RECOMPILE, MAXDOP 1);

**Query Execution Statistics (sys.dm\_exec\_query\_stats)**

More sophisticated version that aggregates data from similar queries using query\_hash column.

Returns one of the execution plans as the sample (the choice of the plan is random)

In [ ]:
;WITH Data
AS
(
    SELECT TOP 50
        qs.query_hash
        ,COUNT(*) as [Plan Count]
        ,MIN(qs.creation_time) AS [Cached Time]
        ,MAX(qs.last_execution_time) AS [Last Exec Time]
        ,SUM(qs.execution_count) AS [Exec Cnt]
        ,SUM(qs.total_logical_reads) AS [Total Reads]
        ,SUM(qs.total_logical_writes) AS [Total Writes]
        ,SUM(qs.total_worker_time / 1000) AS [Total Worker Time]
        ,SUM(qs.total_elapsed_time / 1000) AS [Total Elapsed Time]
        ,SUM(qs.total_rows) AS [Total Rows] 
        ,SUM(qs.total_physical_reads) AS [Total Physical Reads]
        ,SUM(qs.total_grant_kb) AS [Total Grant KB]
        ,SUM(qs.total_used_grant_kb) AS [Total Used Grant KB]
        ,SUM(qs.total_ideal_grant_kb) AS [Total Ideal Grant KB]
        ,SUM(qs.total_columnstore_segment_reads) AS [Total CSI Segments Read] -- SQL Server 2016 SP2+
        ,MAX(qs.max_dop) AS [Max DOP]
        ,SUM(qs.total_spills) AS [Total Spills] -- SQL Server 2016 SP2+
    FROM 
        sys.dm_exec_query_stats qs WITH (NOLOCK)
    GROUP BY
        qs.query_hash
    ORDER BY
        SUM((qs.total_logical_reads + qs.total_logical_writes) /
            qs.execution_count) DESC
)
SELECT 
    d.[Cached Time]
    ,d.[Last Exec Time]
    ,d.[Plan Count]
    ,sql_plan.SQL
    ,TRY_CONVERT(XML,sql_plan.[Query Plan]) AS [Query Plan]
    ,d.[Exec Cnt]
    ,CONVERT(DECIMAL(10,5),
        IIF(datediff(second,d.[Cached Time], d.[Last Exec Time]) = 0,
            NULL,
            1.0 * d.[Exec Cnt] / 
                datediff(second,d.[Cached Time], d.[Last Exec Time])
        )
    ) AS [Exec Per Second]
    ,(d.[Total Reads] + d.[Total Writes]) / d.[Exec Cnt] AS [Avg IO]
    ,(d.[Total Worker Time] / d.[Exec Cnt] / 1000) AS [Avg CPU(ms)]
    ,d.[Total Reads]
    ,d.[Total Writes]
    ,d.[Total Worker Time]
    ,d.[Total Elapsed Time]
    ,d.[Total Rows] 
    ,d.[Total Rows] / d.[Exec Cnt] AS [Avg Rows]
    ,d.[Total Physical Reads]
    ,d.[Total Physical Reads] / d.[Exec Cnt] AS [Avg Physical Reads]
    ,d.[Total Grant KB]
    ,d.[Total Grant KB] / d.[Exec Cnt] AS [Avg Grant KB] 
    ,d.[Total Used Grant KB]
    ,d.[Total Used Grant KB] / d.[Exec Cnt] AS [Avg Used Grant KB] 
    ,d.[Total Ideal Grant KB]
    ,d.[Total Ideal Grant KB] / d.[Exec Cnt] AS [Avg Ideal Grant KB] 
    ,d.[Total CSI Segments Read] -- SQL Server 2016 SP2+
    ,d.[Total CSI Segments Read] / d.[Exec Cnt] AS [AVG CSI Segments Read] -- SQL Server 2016 SP2+
    ,d.[Max DOP]
    ,d.[Total Spills] -- SQL Server 2016 SP2+
    ,d.[Total Spills] / d.[Exec Cnt] AS [Avg Spills] -- SQL Server 2016 SP2+
FROM 
    Data d
        CROSS APPLY
        (
            SELECT TOP 1
                SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
                ((
                    CASE qs.statement_end_offset
                        WHEN -1 THEN DATALENGTH(qt.text)
                        ELSE qs.statement_end_offset
                    END - qs.statement_start_offset)/2)+1
                ) AS SQL
                ,TRY_CONVERT(XML,qp.query_plan) AS [Query Plan]
            FROM
                sys.dm_exec_query_stats qs 
                    OUTER APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
                    OUTER APPLY sys.dm_exec_text_query_plan
                    (
                        qs.plan_handle
                        ,qs.statement_start_offset
                        ,qs.statement_end_offset
                    ) qp
            WHERE
                qs.query_hash = d.query_hash AND ISNULL(qt.text,'') <> ''
        ) sql_plan
ORDER BY
     [Avg IO] DESC
OPTION (RECOMPILE, MAXDOP 1);

**Stored Procedure Execution Statistics (sys.dm\_exec\_proc\_stats)**

Some columns in the output may not work in older SQL Server versions.

In [ ]:
SELECT TOP 50
    IIF (ps.database_id = 32767, 'mssqlsystemresource', DB_NAME(ps.database_id)) AS [DB]
    ,OBJECT_NAME(ps.object_id, IIF(ps.database_id = 32767, 1, ps.database_id)) AS [Proc Name]
    ,ps.type_desc AS [Type]
    ,ps.cached_time AS [Cached Time]
    ,ps.last_execution_time AS [Last Exec Time]
    ,qp.query_plan AS [Plan]
    ,ps.execution_count AS [Exec Count]
    ,CONVERT(DECIMAL(10,5),
        IIF(datediff(second,ps.cached_time, ps.last_execution_time) = 0,
            NULL,
            1.0 * ps.execution_count / 
                datediff(second,ps.cached_time, ps.last_execution_time)
        )
    ) AS [Exec Per Second]
    ,(ps.total_logical_reads + ps.total_logical_writes) / 
        ps.execution_count AS [Avg IO]
    ,(ps.total_worker_time / ps.execution_count / 1000) 
        AS [Avg CPU(ms)]
    ,ps.total_logical_reads AS [Total Reads]
    ,ps.last_logical_reads AS [Last Reads]
    ,ps.total_logical_writes AS [Total Writes]
    ,ps.last_logical_writes AS [Last Writes]
    ,ps.total_worker_time / 1000 AS [Total Worker Time]
    ,ps.last_worker_time / 1000 AS [Last Worker Time]
    ,ps.total_elapsed_time / 1000 AS [Total Elapsed Time]
    ,ps.last_elapsed_time / 1000 AS [Last Elapsed Time]
    ,ps.total_physical_reads AS [Total Physical Reads]
    ,ps.last_physical_reads AS [Last Physical Reads]
    ,ps.total_physical_reads / ps.execution_count AS [Avg Physical Reads]
    ,ps.total_spills AS [Total Spills] -- SQL Server 2016 SP2+
    ,ps.last_spills AS [Last Spills] -- SQL Server 2016 SP2+
    ,(ps.total_spills / ps.execution_count) AS [Avg Spills] -- SQL Server 2016 SP2+
FROM 
    sys.dm_exec_procedure_stats ps WITH (NOLOCK) 
        CROSS APPLY sys.dm_exec_query_plan(ps.plan_handle) qp
ORDER BY
     [Avg IO] DESC


**Getting cached execution plans for individual statements from the module**

Use when previous script does not return the plan. 

Replace the module name in WHERE clause

In [ ]:
SELECT 
    qs.creation_time AS [Cached Time]
    ,qs.last_execution_time AS [Last Exec Time]
    ,SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
    ((
        CASE qs.statement_end_offset
            WHEN -1 THEN DATALENGTH(qt.text)
            ELSE qs.statement_end_offset
        END - qs.statement_start_offset)/2)+1) AS SQL
    ,TRY_CONVERT(XML,qp.query_plan) AS [Query Plan]
    ,CONVERT(DECIMAL(10,5),
        IIF(datediff(second,qs.creation_time, qs.last_execution_time) = 0,
            NULL,
            1.0 * qs.execution_count / 
                datediff(second,qs.creation_time, qs.last_execution_time)
        )
    ) AS [Exec Per Second]
    ,(qs.total_logical_reads + qs.total_logical_writes) / 
        qs.execution_count AS [Avg IO]
    ,(qs.total_worker_time / qs.execution_count / 1000) 
        AS [Avg CPU(ms)]
    ,qs.total_logical_reads AS [Total Reads]
    ,qs.last_logical_reads AS [Last Reads]
    ,qs.total_logical_writes AS [Total Writes]
    ,qs.last_logical_writes AS [Last Writes]
    ,qs.total_worker_time / 1000 AS [Total Worker Time]
    ,qs.last_worker_time / 1000 AS [Last Worker Time]
    ,qs.total_elapsed_time / 1000 AS [Total Elapsed Time]
    ,qs.last_elapsed_time / 1000 AS [Last Elapsed Time]
    ,qs.total_rows AS [Total Rows] 
    ,qs.last_rows AS [Last Rows] 
    ,qs.total_rows / qs.execution_count AS [Avg Rows]
    ,qs.total_physical_reads AS [Total Physical Reads]
    ,qs.last_physical_reads AS [Last Physical Reads]
    ,qs.total_physical_reads / qs.execution_count 
        AS [Avg Physical Reads]
    ,qs.total_grant_kb AS [Total Grant KB]
    ,qs.last_grant_kb AS [Last Grant KB]
    ,(qs.total_grant_kb / qs.execution_count) 
        AS [Avg Grant KB] 
    ,qs.total_used_grant_kb AS [Total Used Grant KB]
    ,qs.last_used_grant_kb AS [Last Used Grant KB]
    ,(qs.total_used_grant_kb / qs.execution_count) 
        AS [Avg Used Grant KB] 
    ,qs.total_ideal_grant_kb AS [Total Ideal Grant KB]
    ,qs.last_ideal_grant_kb AS [Last Ideal Grant KB]
    ,(qs.total_ideal_grant_kb / qs.execution_count) 
        AS [Avg Ideal Grant KB] 
    ,qs.total_columnstore_segment_reads AS [Total CSI Segments Read] -- SQL Server 2016 SP2+
    ,qs.last_columnstore_segment_reads  AS [Last CSI Segments Read] -- SQL Server 2016 SP2+
    ,(qs.total_columnstore_segment_reads / qs.execution_count) AS [AVG CSI Segments Read] -- SQL Server 2016 SP2+
    ,qs.max_dop AS [Max DOP]
    ,qs.total_spills AS [Total Spills] -- SQL Server 2016 SP2+
    ,qs.last_spills AS [Last Spills] -- SQL Server 2016 SP2+
    ,(qs.total_spills / qs.execution_count) AS [Avg Spills] -- SQL Server 2016 SP2+ 
FROM 
    sys.dm_exec_query_stats qs WITH (NOLOCK)
        OUTER APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
        OUTER APPLY sys.dm_exec_text_query_plan
        (
            qs.plan_handle
            ,qs.statement_start_offset
            ,qs.statement_end_offset
        ) qp
WHERE 
    OBJECT_NAME(qt.objectid, qt.dbid) = 'module_name' -- <Add module (SP) name here>
ORDER BY 
    qs.statement_start_offset, qs.statement_end_offset
OPTION (RECOMPILE, MAXDOP 1);


**Getting cached execution plans for scalar UDFs (sys.dm\_exec\_function\_stats)**

Requires SQL Server 2016+

In [ ]:
SELECT TOP 50
    IIF (fs.database_id = 32767, 'mssqlsystemresource', DB_NAME(fs.database_id)) AS [DB]
    ,OBJECT_NAME(fs.object_id, IIF(fs.database_id = 32767, 1, fs.database_id)) AS [Function]
    ,fs.type_desc AS [Type]
    ,fs.cached_time AS [Cached Time]
    ,fs.last_execution_time AS [Last Exec Time]
    ,qp.query_plan AS [Plan]
    ,fs.execution_count AS [Exec Count]
    ,CONVERT(DECIMAL(10,5),
        IIF(datediff(second,fs.cached_time, fs.last_execution_time) = 0,
            NULL,
            1.0 * fs.execution_count / 
                datediff(second,fs.cached_time, fs.last_execution_time)
        )
    ) AS [Exec Per Second]
    ,(fs.total_logical_reads + fs.total_logical_writes) / 
        fs.execution_count AS [Avg IO]
    ,(fs.total_worker_time / fs.execution_count / 1000) AS [Avg CPU(ms)]
    ,fs.total_logical_reads AS [Total Reads]
    ,fs.last_logical_reads AS [Last Reads]
    ,fs.total_logical_writes AS [Total Writes]
    ,fs.last_logical_writes AS [Last Writes]
    ,fs.total_worker_time / 1000 AS [Total Worker Time]
    ,fs.last_worker_time / 1000 AS [Last Worker Time]
    ,fs.total_elapsed_time / 1000 AS [Total Elapsed Time]
    ,fs.last_elapsed_time / 1000 AS [Last Elapsed Time]
    ,fs.total_physical_reads AS [Total Physical Reads]
    ,fs.last_physical_reads AS [Last Physical Reads]
    ,fs.total_physical_reads / fs.execution_count AS [Avg Physical Reads]
FROM 
    sys.dm_exec_function_stats fs WITH (NOLOCK) 
        OUTER APPLY sys.dm_exec_query_plan(fs.plan_handle) qp
ORDER BY
     [Avg IO] DESC
OPTION (RECOMPILE, MAXDOP 1);

**Getting cached execution plans for triggers (sys.dm\_exec\_trigger\_stats)**

In [ ]:
SELECT TOP 50
    IIF (ts.database_id = 32767, 'mssqlsystemresource', DB_NAME(ts.database_id)) AS [DB]
    ,OBJECT_NAME(ts.object_id, IIF(ts.database_id = 32767, 1, ts.database_id)) AS [Function]
    ,ts.type_desc AS [Type]
    ,ts.cached_time AS [Cached Time]
    ,ts.last_execution_time AS [Last Exec Time]
    ,qp.query_plan AS [Plan]
    ,ts.execution_count AS [Exec Count]
    ,CONVERT(DECIMAL(10,5),
        IIF(datediff(second,ts.cached_time, ts.last_execution_time) = 0,
            NULL,
            1.0 * ts.execution_count / 
                datediff(second,ts.cached_time, ts.last_execution_time)
        )
    ) AS [Exec Per Second]
    ,(ts.total_logical_reads + ts.total_logical_writes) / 
        ts.execution_count AS [Avg IO]
    ,(ts.total_worker_time / ts.execution_count / 1000) AS [Avg CPU(ms)]
    ,ts.total_logical_reads AS [Total Reads]
    ,ts.last_logical_reads AS [Last Reads]
    ,ts.total_logical_writes AS [Total Writes]
    ,ts.last_logical_writes AS [Last Writes]
    ,ts.total_worker_time / 1000 AS [Total Worker Time]
    ,ts.last_worker_time / 1000 AS [Last Worker Time]
    ,ts.total_elapsed_time / 1000 AS [Total Elapsed Time]
    ,ts.last_elapsed_time / 1000 AS [Last Elapsed Time]
    ,ts.total_physical_reads AS [Total Physical Reads]
    ,ts.last_physical_reads AS [Last Physical Reads]
    ,ts.total_physical_reads / ts.execution_count AS [Avg Physical Reads]
FROM 
    sys.dm_exec_trigger_stats ts WITH (NOLOCK) 
        OUTER APPLY sys.dm_exec_query_plan(ts.plan_handle) qp
ORDER BY
     [Avg IO] DESC
OPTION (RECOMPILE, MAXDOP 1);